<a href="https://colab.research.google.com/github/Alan-Cheong/IEEE_QW_2020/blob/master/Rubik's_Cube_Gemini_2_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
!pip install kociemba
import copy
import random
# It's generally better to put all imports at the top,
# but we'll import kociemba later inside the try block
# to handle the case where it's not installed.

# --- Cube Representation and Moves ---

class RubiksCube:
    """Represents a 3x3 Rubik's Cube and allows face turns."""

    def __init__(self):
        """Initializes the cube to a solved state."""
        # Colors: W=White (Up), Y=Yellow (Down), B=Blue (Front),
        #         G=Green (Back), R=Red (Left), O=Orange (Right)
        self.faces = {
            'U': [['W'] * 3 for _ in range(3)], # Up face (White)
            'D': [['Y'] * 3 for _ in range(3)], # Down face (Yellow)
            'F': [['B'] * 3 for _ in range(3)], # Front face (Blue)
            'B': [['G'] * 3 for _ in range(3)], # Back face (Green)
            'L': [['R'] * 3 for _ in range(3)], # Left face (Red)
            'R': [['O'] * 3 for _ in range(3)], # Right face (Orange)
        }

    def __str__(self):
        """Provides a simple text-based representation of the unfolded cube."""
        s = ""
        # Print Up face (indented)
        for r in range(3):
            s += "      " + " ".join(self.faces['U'][r]) + "\n"
        s += "\n"
        # Print Left, Front, Right, Back faces side-by-side
        for r in range(3):
            s += " ".join(self.faces['L'][r]) + "  "
            s += " ".join(self.faces['F'][r]) + "  "
            s += " ".join(self.faces['R'][r]) + "  "
            s += " ".join(self.faces['B'][r]) + "\n"
        s += "\n"
        # Print Down face (indented)
        for r in range(3):
            s += "      " + " ".join(self.faces['D'][r]) + "\n"
        return s

    def _rotate_face_clockwise(self, face_key):
        """Rotates the stickers on a single face clockwise."""
        face = self.faces[face_key]
        self.faces[face_key] = [
            [face[2-c][r] for c in range(3)] for r in range(3)
        ]

    def _rotate_face_anticlockwise(self, face_key):
        """Rotates the stickers on a single face anti-clockwise."""
        face = self.faces[face_key]
        self.faces[face_key] = [
            [face[c][2-r] for c in range(3)] for r in range(3)
        ]

    def move(self, move_str):
        """Applies a sequence of moves (e.g., "U R' F2")."""
        moves = move_str.split()
        for move in moves:
            self._apply_single_move(move)

    def _apply_single_move(self, move):
        """Applies a single move notation (e.g., 'U', 'R'', 'F2')."""
        if not move: return
        face_key = move[0]
        is_prime = "'" in move
        is_double = "2" in move

        rotations = 1
        if is_prime:
            rotations = 3
        elif is_double:
            rotations = 2

        for _ in range(rotations):
            self._rotate_clockwise(face_key)

    def _rotate_clockwise(self, face_key):
        """Performs a single clockwise rotation of the specified face."""
        # 1. Rotate the face itself
        self._rotate_face_clockwise(face_key)

        # 2. Rotate the adjacent edge/corner pieces
        temp_row = [None] * 3
        temp_col = [None] * 3

        if face_key == 'U':
            temp_row = self.faces['F'][0][:]
            self.faces['F'][0] = self.faces['L'][0][:]
            self.faces['L'][0] = self.faces['B'][0][:]
            self.faces['B'][0] = self.faces['R'][0][:]
            self.faces['R'][0] = temp_row
        elif face_key == 'D':
            temp_row = self.faces['F'][2][:]
            self.faces['F'][2] = self.faces['R'][2][:]
            self.faces['R'][2] = self.faces['B'][2][:]
            self.faces['B'][2] = self.faces['L'][2][:]
            self.faces['L'][2] = temp_row
        elif face_key == 'F':
            temp_row = self.faces['U'][2][:]
            self.faces['U'][2] = [self.faces['L'][2-r][2] for r in range(3)] # L col 2 (rev) -> U row 2
            for r in range(3): self.faces['L'][r][2] = self.faces['D'][0][r] # D row 0 -> L col 2
            self.faces['D'][0] = [self.faces['R'][2-r][0] for r in range(3)] # R col 0 (rev) -> D row 0
            for r in range(3): self.faces['R'][r][0] = temp_row[r] # Temp (U row 2) -> R col 0
        elif face_key == 'B':
            temp_row = self.faces['U'][0][:]
            self.faces['U'][0] = [self.faces['R'][2-r][2] for r in range(3)] # R col 2 (rev) -> U row 0
            for r in range(3): self.faces['R'][r][2] = self.faces['D'][2][r] # D row 2 -> R col 2
            self.faces['D'][2] = [self.faces['L'][2-r][0] for r in range(3)] # L col 0 (rev) -> D row 2
            for r in range(3): self.faces['L'][r][0] = temp_row[r] # Temp (U row 0) -> L col 0

        # --- Logic based on refining the *original* code's implementation ---
        elif face_key == 'L':
            # 1. Save Up col 0
            temp_col = [self.faces['U'][r][0] for r in range(3)]
            # 2. Back col 2 (reversed) -> Up col 0
            for r in range(3): self.faces['U'][r][0] = self.faces['B'][2-r][2]
            # 3. Down col 0 (reversed) -> Back col 2
            for r in range(3): self.faces['B'][r][2] = self.faces['D'][2-r][0] # Corrected from original
            # 4. Front col 0 -> Down col 0
            for r in range(3): self.faces['D'][r][0] = self.faces['F'][r][0]
            # 5. Temp (Up col 0) -> Front col 0
            for r in range(3): self.faces['F'][r][0] = temp_col[r] # Corrected from original

        elif face_key == 'R':
            # 1. Save Up col 2
            temp_col = [self.faces['U'][r][2] for r in range(3)]
            # 2. Front col 2 -> Up col 2
            for r in range(3): self.faces['U'][r][2] = self.faces['F'][r][2]
            # 3. Down col 2 -> Front col 2
            for r in range(3): self.faces['F'][r][2] = self.faces['D'][r][2]
            # 4. Back col 0 (reversed) -> Down col 2
            for r in range(3): self.faces['D'][r][2] = self.faces['B'][2-r][0]
            # 5. Temp (Up col 2) -> Back col 0 (reversed)
            for r in range(3): self.faces['B'][r][0] = temp_col[2-r] # Original logic looked correct here
        # --- End of refined original logic ---

    def get_state_string(self):
        """
        Generates a cube state string compatible with some solvers (like kociemba).
        Order: UUUUUUUUU RRRRRRRRR FFFFFFFFF DDDDDDDDD LLLLLLLLL BBBBBBBBB
        """
        order = ['U', 'R', 'F', 'D', 'L', 'B']
        state_str = ""
        for face_key in order:
            for r in range(3):
                state_str += "".join(self.faces[face_key][r]) # Simplified join
        return state_str

    def scramble(self, num_moves=20):
        """Applies a random sequence of moves."""
        faces = ['U', 'D', 'L', 'R', 'F', 'B']
        modifiers = ['', "'", '2']
        scramble_moves = []
        last_face = None
        for _ in range(num_moves):
            while True:
                face = random.choice(faces)
                if face != last_face:
                    break
            mod = random.choice(modifiers)
            scramble_moves.append(face + mod)
            last_face = face
        scramble_str = " ".join(scramble_moves)
        print(f"Scramble: {scramble_str}")
        self.move(scramble_str)
        return scramble_str

    # --- Solving Placeholder ---
    def solve(self):
        """
        Placeholder for a solving algorithm.
        Implementing this is the complex part.
        For practical use, call an external library.
        """
        print("\n--- Solving (Using External Library Recommended) ---")
        print("To actually solve, use a library like 'kociemba'.")
        print("See example usage below main execution block.")
        pass

# --- Main Execution ---
if __name__ == "__main__":
    # Code inside this block should be indented by 4 spaces
    my_cube = RubiksCube()
    print("--- Solved Cube ---")
    print(my_cube)

    # Scramble the cube
    print("\n--- Scrambling ---")
    my_cube.scramble(20) # Apply 20 random moves
    print("\n--- Scrambled Cube ---")
    print(my_cube)

    # Get the state string for potential solver input
    cube_state_string = my_cube.get_state_string()
    print(f"\nCube state string: {cube_state_string}")

    # Placeholder call to solve (doesn't actually solve here)
    # my_cube.solve() # Commenting this out as we try the real solve below

    print("\n--- Using the 'kociemba' library for solving ---")
    # REMINDER: You need to install the library first, e.g., using:
    # pip install kociemba
    # OR
    # python -m pip install kociemba
    # Do this in your terminal/command prompt BEFORE running the script.

    # --- Example using the 'kociemba' library ---
    # 'try' should be indented 4 spaces (inside the 'if __name__ ...' block)
    try:
        # Code inside 'try' should be indented 8 spaces
        import kociemba # Import the library here

        print("\n--- Attempting to solve with kociemba library ---")
        scrambled_state = my_cube.get_state_string()
        print(f"Solving state: {scrambled_state}")

        # The kociemba.solve() function takes the state string
        # It returns the solution sequence as a string
        solution_moves = kociemba.solve(scrambled_state)

        print(f"Solution found: {solution_moves}")

        # Apply the solution to get back to solved state
        print("\n--- Applying Solution ---")
        my_cube.move(solution_moves)
        print("\n--- Cube After Applying Solution ---")
        print(my_cube)
        print("Cube should now be solved!")

    # 'except' must align with 'try' (4 spaces)
    except ImportError:
        # Code inside 'except' should be indented 8 spaces
        print("\nERROR: 'kociemba' library not found.")
        print("Please install it using: pip install kociemba")

    # 'except' must align with 'try' (4 spaces)
    except Exception as e:
        # Code inside 'except' should be indented 8 spaces
        print(f"\nAn error occurred during solving: {e}")
        print("The cube state string might be invalid (check representation/scramble).")
        print("Ensure the cube state string has exactly 54 characters.")

--- Solved Cube ---
      W W W
      W W W
      W W W

R R R  B B B  O O O  G G G
R R R  B B B  O O O  G G G
R R R  B B B  O O O  G G G

      Y Y Y
      Y Y Y
      Y Y Y


--- Scrambling ---
Scramble: U' L2 B' R L D' U' B F2 B' L2 D L2 R' U2 D2 F L U L'

--- Scrambled Cube ---
      W W R
      W W R
      Y G R

B G O  O B G  W Y G  Y O R
R R O  O B Y  B O W  G G O
R R G  W W Y  B G W  G B B

      B B O
      Y Y Y
      Y R O


Cube state string: WWRWWRYGRWYGBOWBGWOBGOBYWWYBBOYYYYROBGORRORRGYORGGOGBB

--- Using the 'kociemba' library for solving ---

--- Attempting to solve with kociemba library ---
Solving state: WWRWWRYGRWYGBOWBGWOBGOBYWWYBBOYYYYROBGORRORRGYORGGOGBB

An error occurred during solving: Error. Probably cubestring is invalid
The cube state string might be invalid (check representation/scramble).
Ensure the cube state string has exactly 54 characters.
